In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [5]:
# Santa Hill Climbing 2019

import numba
from numba import njit
from numba.typed import Dict
import numpy as np
from time import time
from os import path
import csv

In [3]:
import scipy.special
import matplotlib.pyplot as plt

In [4]:
fpath = r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')
data_choices = data.values

fpath = r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\sample_submission.csv'
submission = pd.read_csv(fpath, index_col='family_id')

In [12]:
# const

DBL_MAX = 1e+308

N_FAMILES = 5000
N_DAYS = 100
N_CHOICES = 10
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

MAX_DIFF = 300
MAX_DIFF2 = MAX_DIFF * 2
MAX_FAMILY_PER_DAY = 200

In [10]:
DATA_DIR = "C:/Users/Solomonzhs/Desktop/Kaggle/santa-workshop-tour-2019/"

In [11]:
def build_cost_lut(family_size, family_choice):
    pref_cost = np.empty((N_FAMILES, N_DAYS), dtype=np.float64)
    acc1_cost = np.empty((MAX_DIFF2,), dtype=np.float64)
    acc_cost = np.empty((MAX_DIFF2, MAX_DIFF2), dtype=np.float64)
    penalty = np.empty((MAX_DIFF2, ), dtype=np.float64)

    for i in range(N_FAMILES):
        # preference cost
        n = family_size[i]
        pref_cost[i][:] = 500 + 36 * n + 398 * n
        pref_cost[i][family_choice[i][0]] = 0
        pref_cost[i][family_choice[i][1]] = 50
        pref_cost[i][family_choice[i][2]] = 50 + 9 * n
        pref_cost[i][family_choice[i][3]] = 100 + 9 * n
        pref_cost[i][family_choice[i][4]] = 200 + 9 * n
        pref_cost[i][family_choice[i][5]] = 200 + 18 * n        
        pref_cost[i][family_choice[i][6]] = 300 + 18 * n
        pref_cost[i][family_choice[i][7]] = 300 + 36 * n
        pref_cost[i][family_choice[i][8]] = 400 + 36 * n
        pref_cost[i][family_choice[i][9]] = 500 + 36 * n + 199 * n
        
    for i in range(MAX_DIFF2):
        # accounting cost
        acc1_cost[i] = max(0, (i - 125.0) / 400.0 * i ** 0.5)
        for j in range(MAX_DIFF2):
            diff = abs(j - MAX_DIFF)
            acc_cost[i][j] = max(0, (i - 125.0) / 400.0 * i ** (0.5 + diff / 50.0))

        # constraint penalty
        if i > MAX_OCCUPANCY:
            penalty[i] = 60 * (i - MAX_OCCUPANCY + 1) ** 1.2
        elif i < MIN_OCCUPANCY:
            penalty[i] = 60 * (MIN_OCCUPANCY - i + 1) ** 1.2
        else:
            penalty[i] = 0

    return pref_cost, acc1_cost, acc_cost, penalty

In [14]:
def build_global_data(data_dir):
    # family data
    family_choice = np.empty((N_FAMILES, N_CHOICES), dtype=np.int32)
    family_size = np.empty((N_FAMILES,), dtype=np.int32)

    with open(path.join(data_dir, "family_data.csv"), "r") as f:
        reader = csv.reader(f)
        next(reader, None)
        for row in reader:
            i = int(row[0])
            choices = [int(c) - 1 for c in row[1:N_CHOICES+1]]
            members = int(row[N_CHOICES+1])
            family_size[i] = members
            family_choice[i] = choices
    # cost lut
    pref_cost, acc1_cost, acc_cost, penalty = build_cost_lut(family_size, family_choice)
    return pref_cost, acc1_cost, acc_cost, penalty, family_choice, family_size

PREF_COST, ACC1_COST, ACC_COST, PENALTY, FAMILY_CHOICES, FAMILY_SIZE = build_global_data(DATA_DIR)

In [15]:
PREF_COST, ACC1_COST, ACC_COST, PENALTY, FAMILY_CHOICES, FAMILY_SIZE = build_global_data(DATA_DIR)

In [19]:
def insertion_cost(f_days, n_days, family_id, d_to, constraint_weight=1):
    """ insertion cost for greedy method
    """
    # preference cost
    pref_cost = PREF_COST[family_id][d_to]

    # accounting cost
    n = n_days[d_to]
    n_new = n + FAMILY_SIZE[family_id]
    if d_to == N_DAYS - 1:
        old_acc_cost = ACC1_COST[n]
        new_acc_cost = ACC1_COST[n_new]
    else:
        old_diff = n - n_days[d_to + 1]
        new_diff = n_new - n_days[d_to + 1]
        old_acc_cost = ACC_COST[n][old_diff + MAX_DIFF]
        new_acc_cost = ACC_COST[n_new][new_diff + MAX_DIFF]

    # constraints penalty
    old_penalty = PENALTY[n]
    new_penalty = PENALTY[n_new]

    return pref_cost + (new_acc_cost - old_acc_cost) + (new_penalty - old_penalty) * constraint_weight

In [20]:
def total_cost(f_days, c_days, n_days, constraint_weight=1):
    """ cost function for santa-2019
    """
    
    # preference cost
    pref_cost = 0
    for i in range(N_DAYS):
        cost = 0
        for j in range(c_days[i]):
            cost += PREF_COST[f_days[i][j]][i]
        pref_cost += cost

    # accounting cost and constraints penalty
    acc_cost = ACC1_COST[n_days[N_DAYS - 1]]
    penalty = PENALTY[n_days[N_DAYS - 1]]
    for i in range(N_DAYS - 1):
        n = n_days[i]
        diff = n - n_days[i + 1]
        acc_cost += ACC_COST[n][diff + MAX_DIFF]
        penalty += PENALTY[n]
    return pref_cost + acc_cost + penalty * constraint_weight

In [21]:
def day_insert(f_days, c_days, n_days, day, family_id):
    """ insert family_id to day
    """
    f_days[day][c_days[day]] = family_id
    c_days[day] += 1
    n_days[day] += FAMILY_SIZE[family_id]

In [22]:
def day_remove(f_days, c_days, n_days, day, pos):
    """ remove family_id from day
    """
    family_id = f_days[day][pos]
    f_days[day][pos] = f_days[day][c_days[day]-1]
    c_days[day] -= 1
    n_days[day] -= FAMILY_SIZE[family_id]
    return family_id

In [ ]:
family_size_dict = data[['n_people']].to_dict()['n_people']

cols = [f'choice_{i}' for i in range(10)]
choice_dict = data[cols].to_dict()

N_DAYS = 100
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

# from 100 to 1
days = list(range(N_DAYS,0,-1))

In [ ]:
def calc_family_costs(family):
    assigned_day = family['assigned_day']
    number_member = family['n_people']
    if assigned_day == family['choice_0']:
        penalty = 0
    elif assigned_day == family['choice_1']:
        penalty = 50
    elif assigned_day == family['choice_2']:
        penalty = 50 + 9 * number_member
    elif assigned_day == family['choice_3']:
        penalty = 100 + 9 * number_member
    elif assigned_day == family['choice_4']:
        penalty = 200 + 9 * number_member
    elif assigned_day == family['choice_5']:
        penalty = 200 + 18 * number_member
    elif assigned_day == family['choice_6']:
        penalty = 300 + 18 * number_member
    elif assigned_day == family['choice_7']:
        penalty = 300 + 36 * number_member
    elif assigned_day == family['choice_8']:
        penalty = 400 + 36 * number_member
    elif assigned_day == family['choice_9']:
        penalty = 500 + 36 * number_member + 199 * number_member
    else:
        penalty = 500 + 36 * number_member + 398 * number_member
    return penalty

In [ ]:
def cost_function(prediction):

    penalty = 0

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = {k:0 for k in days}
    
    # Looping over each family; d is the day for each family f
    for f, d in enumerate(prediction):

        # Using our lookup dictionaries to make simpler variable names
        n = family_size_dict[f]
        choice_0 = choice_dict['choice_0'][f]
        choice_1 = choice_dict['choice_1'][f]
        choice_2 = choice_dict['choice_2'][f]
        choice_3 = choice_dict['choice_3'][f]
        choice_4 = choice_dict['choice_4'][f]
        choice_5 = choice_dict['choice_5'][f]
        choice_6 = choice_dict['choice_6'][f]
        choice_7 = choice_dict['choice_7'][f]
        choice_8 = choice_dict['choice_8'][f]
        choice_9 = choice_dict['choice_9'][f]

        # add the family member count to the daily occupancy
        daily_occupancy[d] += n

        # Calculate the penalty for not getting top preference
        if d == choice_0:
            penalty += 0
        elif d == choice_1:
            penalty += 50
        elif d == choice_2:
            penalty += 50 + 9 * n
        elif d == choice_3:
            penalty += 100 + 9 * n
        elif d == choice_4:
            penalty += 200 + 9 * n
        elif d == choice_5:
            penalty += 200 + 18 * n
        elif d == choice_6:
            penalty += 300 + 18 * n
        elif d == choice_7:
            penalty += 300 + 36 * n
        elif d == choice_8:
            penalty += 400 + 36 * n
        elif d == choice_9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n

    # for each date, check total occupancy
    #  (using soft constraints instead of hard constraints)
    for _, v in daily_occupancy.items():
        if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
            penalty += 100000000

    # Calculate the accounting cost
    # The first day (day 100) is treated special
    accounting_cost = (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5)
    # using the max function because the soft constraints might allow occupancy to dip below 125
    accounting_cost = max(0, accounting_cost)
    
    # Loop over the rest of the days, keeping track of previous count
    yesterday_count = daily_occupancy[days[0]]
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (daily_occupancy[day]-125.0) / 400.0 * daily_occupancy[day]**(0.5 + diff / 50.0))
        yesterday_count = today_count

    penalty += accounting_cost
    
    return penalty

In [ ]:
# Start with the sample submission values
best = submission['assigned_day'].tolist()
start_score = cost_function(best)

new = best.copy()
# loop over each family
for fam_id, _ in enumerate(best):
    # loop over each family choice
    for pick in range(10):
        day = choice_dict[f'choice_{pick}'][fam_id]
        temp = new.copy()
        temp[fam_id] = day # add in the new pick
        if cost_function(temp) < start_score:
            new = temp.copy()
            start_score = cost_function(new)

submission['assigned_day'] = new
score = cost_function(new)
submission.to_csv(f'submission_{score}.csv')
print(f'Score: {score}')

In [ ]:
jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [ ]:
def plot_results(data, name):
    x = data.columns
    y = data.loc[name]
    plt.plot(x, y, 'ro')
    plt.grid()
    plt.title(name)
    plt.xlabel('steps')
    plt.ylabel('value')
    plt.show()

In [ ]:
def check_day(data, day):
    group_data = data.groupby('assigned_day').sum()['n_people'].to_frame()
    if (125 <= group_data.loc[day, 'n_people']) & (group_data.loc[day, 'n_people'] <= 300):
        return True
    else:
        return False

In [ ]:
for i in range(N_DAYS):
    data.loc[i*50:(i+1)*50-1, 'assigned_day'] = i+1
data['assigned_day'] = data['assigned_day'].astype(int)

In [ ]:
family_id = 100
cost_function(data)

In [ ]:
data['penalty_cost'] = data.apply(cost_function, axis=1)

In [ ]:
print('Total costs:', data['penalty_cost'].sum())

In [ ]:
def check_swap_day(data, family, choice):
    data_copy = data.copy()
    data_copy.loc[family, 'assigned_day'] = data_copy.loc[family, 'choice_'+str(choice)]
    data_copy.loc[family, 'penalty_cost'] = calc_family_costs(data_copy.iloc[family])
    
    penalty_before = data.loc[family, 'penalty_cost']
    
    penalty_after = data_copy.loc[family, 'penalty_cost']
    
    # Check conditions
    day_before = check_day(data_copy, data.loc[family, 'assigned_day'])
    day_after = check_day(data_copy, data_copy.loc[family, 'assigned_day'])

    if(day_before==True and day_after==True):
        improvement = (penalty_before-penalty_after)
    else:
        improvement = -1
    
    return improvement

In [ ]:
family_id = 386
check_swap_day(data, family_id, 0)

In [ ]:
def check_swap_family(data, family, choice):
    family1 = family
    day_family1 = data.loc[family1, 'assigned_day']
    penalty1 = data.loc[family1, 'penalty_cost']
    member_family1 = data.loc[family1, 'n_people']
    
    data = data.reset_index()
    day_member_list = data.groupby('assigned_day')['family_id'].apply(list).to_frame()
    
    improvements = {}
    for member in day_member_list.loc[data.loc[family1, 'choice_'+str(choice)], 'family_id']:
        family2 = member
        day_family2 = data.loc[family2, 'assigned_day']
        member_family2 = data.loc[family2, 'n_people']
        penalty2 = data.loc[family2, 'penalty_cost']
        
        # simulate the swap with another family
        data_copy = data.copy()
        data_copy.loc[family2, 'assigned_day'] = data_copy.loc[family1, 'assigned_day']
        data_copy.loc[family1, 'assigned_day'] = data_copy.loc[family1, 'choice_'+str(choice)]
        # calc the new penalty cost for both families
        new_penalty1 = calc_family_costs(data_copy.iloc[family1])
        new_penalty2 = calc_family_costs(data_copy.iloc[family2])
        # check both days before and after swaping
        day_before = check_day(data_copy, data.loc[family1, 'assigned_day'])
        day_after = check_day(data_copy, data_copy.loc[family1, 'choice_'+str(choice)])
        
        # calc the accounting costs before and after swaping
        accounting_before = calc_accounting_cost(data)
        accounting_after = calc_accounting_cost(data_copy)
        if(day_before==True and day_after==True):
            improvement = (penalty1-new_penalty1) + (penalty2-new_penalty2) + (accounting_before-accounting_after)
        else:
            improvement = -1
        improvements.update({member:improvement})
   
    maximum = max(zip(improvements.values(), improvements.keys()))
    family_swap = maximum[1]
    return improvement, family_swap

In [ ]:
family_id = 386
check_swap_family(data, family_id, 0)

# New- Solver Min Cost

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
NUMBER_DAYS = 100
NUMBER_FAMILIES = 5000
MAX_BEST_CHOICE = 5

In [5]:
data = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\family_data.csv')

submission = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\sample_submission.csv')

In [7]:
assigned_days = submission['assigned_day'].values
columns = data.columns[1:11]
DESIRED = data[columns].values

In [8]:
COST_PER_FAMILY        = [0,50,50,100,200,200,300,300,400,500]
COST_PER_FAMILY_MEMBER = [0, 0, 9,  9,  9, 18, 18, 36, 36,235]
N_PEOPLE = data['n_people'].astype(int).values

In [9]:
def get_daily_occupancy(assigned_days):
    daily_occupancy = np.zeros(100, np.int32)
    for i, r in enumerate(assigned_days):
        daily_occupancy[r-1] += N_PEOPLE[i]
    return daily_occupancy

In [16]:
def cost_function(prediction):
    N_DAYS = NUMBER_DAYS
    MAX_OCCUPANCY = 300
    MIN_OCCUPANCY = 125
    penalty = 0
    days = list(range(N_DAYS,0,-1))
    tmp = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\family_data.csv', index_col='family_id')
    family_size_dict = tmp[['n_people']].to_dict()['n_people']

    cols = [f'choice_{i}' for i in range(10)]
    choice_dict = tmp[cols].to_dict()

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = {k:0 for k in days}
    
    # Looping over each family; d is the day for each family f
    for f, d in enumerate(prediction):
        # Using our lookup dictionaries to make simpler variable names
        n = family_size_dict[f]

        choice_0 = choice_dict['choice_0'][f]
        choice_1 = choice_dict['choice_1'][f]
        choice_2 = choice_dict['choice_2'][f]
        choice_3 = choice_dict['choice_3'][f]
        choice_4 = choice_dict['choice_4'][f]
        choice_5 = choice_dict['choice_5'][f]
        choice_6 = choice_dict['choice_6'][f]
        choice_7 = choice_dict['choice_7'][f]
        choice_8 = choice_dict['choice_8'][f]
        choice_9 = choice_dict['choice_9'][f]

        # add the family member count to the daily occupancy
        daily_occupancy[d] += n

        # Calculate the penalty for not getting top preference
        if d == choice_0:
            penalty += 0
        elif d == choice_1:
            penalty += 50
        elif d == choice_2:
            penalty += 50 + 9 * n
        elif d == choice_3:
            penalty += 100 + 9 * n
        elif d == choice_4:
            penalty += 200 + 9 * n
        elif d == choice_5:
            penalty += 200 + 18 * n
        elif d == choice_6:
            penalty += 300 + 18 * n
        elif d == choice_7:
            penalty += 300 + 36 * n
        elif d == choice_8:
            penalty += 400 + 36 * n
        elif d == choice_9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n

    # for each date, check total occupancy
    #  (using soft constraints instead of hard constraints)
    for _, v in daily_occupancy.items():
        if v > MAX_OCCUPANCY or v < MIN_OCCUPANCY:
            penalty += 100000000

    # Calculate the accounting cost
    # The first day (day 100) is treated special
    # using the max function because the soft constraints might allow occupancy to dip below 125
    accounting_cost = max(0, (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5))
    # Loop over the rest of the days, keeping track of previous count
    yesterday_count = daily_occupancy[days[0]]
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (today_count-125.0) / 400.0 * today_count**(0.5 + diff / 50.0))
        yesterday_count = today_count

    return penalty, accounting_cost, penalty + accounting_cost

In [17]:
data.n_people.unique()

array([4, 3, 2, 5, 7, 6, 8], dtype=int64)

In [18]:
from ortools.graph import pywrapgraph

for num_members in range(2, 9): # Families have minimum 2 and maximum 8 members
    daily_occupancy = get_daily_occupancy(assigned_days)
    fids = np.where(N_PEOPLE == num_members)[0]

    PCOSTM = {}
    for fid in range(NUMBER_FAMILIES):
        if fid in fids:
            for i in range(MAX_BEST_CHOICE):
                PCOSTM[fid, DESIRED[fid][i]-1] = COST_PER_FAMILY[i] + N_PEOPLE[fid] * COST_PER_FAMILY_MEMBER[i]
        else:
            daily_occupancy[assigned_days[fid]-1] -= N_PEOPLE[fid]

    offset = fids.shape[0]
    solver = pywrapgraph.SimpleMinCostFlow()
    for day in range(NUMBER_DAYS):
        solver.SetNodeSupply(offset+day, int(daily_occupancy[day]//num_members))

    for i in range(offset):
        fid = fids[i]
        solver.SetNodeSupply(i, -1)
        for j in range(MAX_BEST_CHOICE):
            day = DESIRED[fid][j]-1
            solver.AddArcWithCapacityAndUnitCost(int(offset+day), i, 1, int(PCOSTM[fid, day]))
    solver.SolveMaxFlowWithMinCost()

    for i in range(solver.NumArcs()):
        if solver.Flow(i) > 0:
            assigned_days[fids[solver.Head(i)]] = solver.Tail(i) - offset + 1
    print(cost_function(assigned_days))

(9756955, 1941.2534888812897, 9758896.253488882)
(8184241, 2253.902261563777, 8186494.902261564)
(5322825, 6564.762943264543, 5329389.762943264)
(3187985, 7082.630899220902, 3195067.6308992207)
(1825615, 17218.631610250108, 1842833.63161025)
(935298, 17413.745930630026, 952711.74593063)
(422846, 16587.60956684803, 439433.609566848)


# New 2 - PyTorch 

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from numba import njit
MAX_CHOICE = 5

In [23]:
data = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\family_data.csv', index_col='family_id')
data_choices = data.values
submission = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\sample_submission.csv', index_col='family_id')

In [24]:
dummies = []
for i in range(MAX_CHOICE):
    tmp = pd.get_dummies(data[f'choice_{i}']).values*(data['n_people'].values.reshape(-1,1))
    dummies.append((
        np.concatenate([tmp, tmp[:, -1].reshape(-1,1)], axis=1)
                   ).reshape(5000, 101, 1))
dummies = np.concatenate(dummies, axis=2)
dummies = np.swapaxes(dummies, 1, 2)

penalties = {n: [0, 50, 50 + 9 * n, 100 + 9 * n, 200 + 9 * n, 200 + 18 * n, 300 + 18 * n, 300 + 36 * n, 400 + 36 * n, 500 + 36 * n + 199 * n] for n in np.unique(data['n_people'])}

mat = []
for i in range(5000):
    n = data.iloc[i]['n_people']
    mat.append(penalties[n][:MAX_CHOICE])
mat = np.array(mat)

In [25]:
def create_init(initial_sub):
    
    fam_choices = data
    a = pd.merge(initial_sub, fam_choices, on='family_id')

    initial_choices = []
    for i in range(MAX_CHOICE):
        initial_choices.append(((a[f'choice_{i}'] == a['assigned_day'])).values.reshape(-1,1))
    initial_choices = np.concatenate(initial_choices, axis=1)
    initial_choices = torch.tensor(
       initial_choices*10
        , dtype=torch.float32)#.cuda()
    return initial_choices

initial_sub = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\sample_submission.csv')
initial_choices = create_init(initial_sub)

In [28]:
family_sizes = data.n_people.values.astype(np.int8)
cost_dict = {0:  [  0,  0],
             1:  [ 50,  0],
             2:  [ 50,  9],
             3:  [100,  9],
             4:  [200,  9],
             5:  [200, 18],
             6:  [300, 18],
             7:  [300, 36],
             8:  [400, 36],
             9:  [500, 36 + 199],
             10: [500, 36 + 398],
            }

In [29]:
def cost(choice, members, cost_dict):
    x = cost_dict[choice]
    return x[0] + members * x[1]
all_costs = {k: pd.Series([cost(k, x, cost_dict) for x in range(2,9)], index=range(2,9)) for k in cost_dict.keys()}
df_all_costs = pd.DataFrame(all_costs)

family_cost_matrix = np.zeros((100,len(family_sizes))) # Cost for each family for each day.

for i, el in enumerate(family_sizes):
    family_cost_matrix[:, i] += all_costs[10][el] # populate each day with the max cost
    for j, choice in enumerate(data.drop("n_people",axis=1).values[i,:]):
        family_cost_matrix[choice-1, i] = all_costs[j][el]
        
def accounting(today, previous):
    return ((today - 125) / 400 ) * today ** (.5 + (abs(today - previous) / 50))

acc_costs = np.zeros([176,176])

for i, x in enumerate(range(125,300+1)):
    for j, y in enumerate(range(125,300+1)):
        acc_costs[i,j] = accounting(x,y)

In [30]:
@njit(fastmath=True)
def cost_function(prediction, family_size, family_cost_matrix, accounting_cost_matrix):
    N_DAYS = 100
    MAX_OCCUPANCY = 300
    MIN_OCCUPANCY = 125
    penalty = 0
    accounting_cost = 0
    max_occ = False
    
    daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int16)
    for i, (pred, n) in enumerate(zip(prediction, family_size)):
        daily_occupancy[pred - 1] += n
        penalty += family_cost_matrix[pred - 1, i]
        
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_next = daily_occupancy[day + 1]
        n = daily_occupancy[day]
        max_occ += MIN_OCCUPANCY > n
        max_occ += MAX_OCCUPANCY < n
        accounting_cost += accounting_cost_matrix[n-MIN_OCCUPANCY, n_next-MIN_OCCUPANCY]
    if max_occ: 
        return 1e11
    return penalty+accounting_cost

In [31]:
cost_function(initial_sub['assigned_day'].values, family_sizes, family_cost_matrix, acc_costs)

10641498.403135022

In [32]:
class Model(nn.Module):
    def __init__(self, mat, dummies):
        super().__init__()
        self.mat = torch.from_numpy(mat).type(torch.int16)#.cuda()
        self.dummies = torch.from_numpy(dummies).type(torch.float32)#.cuda()
        self.weight = torch.nn.Parameter(data=torch.Tensor(5000, MAX_CHOICE).type(torch.float32)#.cuda()
                                         , requires_grad=True)
        self.weight.data.copy_(initial_choices)
        
    def forward(self):
        prob = F.softmax(self.weight,dim=1)
        
        x = (prob * self.mat).sum()
        
        daily_occupancy = torch.zeros(101, dtype=torch.float32)#.cuda()
        for i in range(MAX_CHOICE):
            daily_occupancy += (prob[:, i]@self.dummies[:, i, :])
        
        diff = torch.abs(daily_occupancy[:-1] - daily_occupancy[1:])
        daily_occupancy = daily_occupancy[:-1]
        y = (
            torch.relu(daily_occupancy-125.0) / 400.0 * daily_occupancy**(0.5 + diff / 50.0)
        ).sum() 
        
        v = ((torch.relu(125-daily_occupancy))**2+(torch.relu(daily_occupancy-300))**2).sum()
        
        entropy_loss = -1.0 * (prob * F.log_softmax(self.weight, dim=1)).sum()
        return  x, y, v*10000, entropy_loss

In [33]:
model = Model(mat, dummies)
best_score = 10e10
best_pos = None
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

In [34]:
for epoch in tqdm.tqdm_notebook(range(1_001)):
    optimizer.zero_grad()
    x, y, v, ent = model()
    loss = x + y + v + 0*ent
    loss.backward()
    optimizer.step()
    
    pos = model.weight.argmax(1).cpu().numpy()
    pred = []
    for i in range(5000):
        pred.append(data_choices[i, pos[i]])
    pred = np.array(pred)
    score = cost_function(pred, family_sizes, family_cost_matrix, acc_costs)
    if (score < best_score):
        best_score = score
        best_pos = pred
        print(best_score)
        submission['assigned_day'] = best_pos
        submission.to_csv(f'submission.csv')
    if epoch % 1000 == 0:
            x = np.round(x.item(),1)
            y = np.round(y.item(),1)
            print(f'{epoch}\t{x}\t{y}    \t{np.round(score, 2)}')

0	512341.5	inf    	100000000000.0
1000	nan	nan    	100000000000.0



# New 3 - Modeling Base Logic 

In [35]:
import numpy as np
import pandas as pd
import scipy.special
import matplotlib.pyplot as plt
import os

In [36]:
data = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\family_data.csv', index_col='family_id')
data_choices = data.values
submission = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Kaggle\santa-workshop-tour-2019\sample_submission.csv', index_col='family_id')

In [37]:
num_days = 100
lower = 125
upper = 300
days = list(range(num_days, 0, -1))

In [38]:
def calc_family_costs(family):
    assigned_day = family['assigned_day']
    number_member = family['n_people']
    if assigned_day == family['choice_0']:
        penalty = 0
    elif assigned_day == family['choice_1']:
        penalty = 50
    elif assigned_day == family['choice_2']:
        penalty = 50 + 9 * number_member
    elif assigned_day == family['choice_3']:
        penalty = 100 + 9 * number_member
    elif assigned_day == family['choice_4']:
        penalty = 200 + 9 * number_member
    elif assigned_day == family['choice_5']:
        penalty = 200 + 18 * number_member
    elif assigned_day == family['choice_6']:
        penalty = 300 + 18 * number_member
    elif assigned_day == family['choice_7']:
        penalty = 300 + 36 * number_member
    elif assigned_day == family['choice_8']:
        penalty = 400 + 36 * number_member
    elif assigned_day == family['choice_9']:
        penalty = 500 + 36 * number_member + 199 * number_member
    else:
        penalty = 500 + 36 * number_member + 398 * number_member
    return penalty

In [42]:
def calc_accounting_cost(data):
    accounting_cost = 0
    daily_occupancy = {k:0 for k in days}
    family_size_dict = data[['n_people']].to_dict()['n_people']
    for f, d in enumerate(data['assigned_day']):
        n = family_size_dict[f]
        daily_occupancy[d] += n

    accounting_cost = (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5)
    accounting_cost = max(0, accounting_cost)
    
    yesterday_count = daily_occupancy[days[0]]
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (daily_occupancy[day]-125.0) / 400.0 * daily_occupancy[day]**(0.5 + diff / 50.0))
        yesterday_count = today_count
    return accounting_cost

In [43]:
def plot_results(data, name):
    x = data.columns
    y = data.loc[name]
    plt.plot(x, y, 'ro')
    plt.grid()
    plt.title(name)
    plt.xlabel('steps')
    plt.ylabel('value')
    plt.show()

In [44]:
def check_day(data, day):
    group_data = data.groupby('assigned_day').sum()['n_people'].to_frame()
    if (125 <= group_data.loc[day, 'n_people']) & (group_data.loc[day, 'n_people'] <= 300):
        return True
    else:
        return False

In [45]:
for i in range(num_days):
    data.loc[i*50:(i+1)*50-1, 'assigned_day'] = i+1
data['assigned_day'] = data['assigned_day'].astype(int)

In [48]:
data['assigned_day'] = submission['assigned_day']

In [49]:
family_id = 100
calc_family_costs(data.iloc[family_id])

3104

In [50]:
data['penalty_cost'] = data.apply(calc_family_costs, axis=1)

In [51]:
acc_costs = calc_accounting_cost(data)
acc_costs

1907.4031350226594

In [52]:
print('Total costs:', data['penalty_cost'].sum()+ acc_costs)

Total costs: 10641498.403135022


In [53]:
def check_swap_day(data, family, choice):
    data_copy = data.copy()
    data_copy.loc[family, 'assigned_day'] = data_copy.loc[family, 'choice_'+str(choice)]
    data_copy.loc[family, 'penalty_cost'] = calc_family_costs(data_copy.iloc[family])
    
    penalty_before = data.loc[family, 'penalty_cost']
    accounting_before = calc_accounting_cost(data)
    
    penalty_after = data_copy.loc[family, 'penalty_cost']
    accounting_after = calc_accounting_cost(data_copy)
    
    # Check conditions
    day_before = check_day(data_copy, data.loc[family, 'assigned_day'])
    day_after = check_day(data_copy, data_copy.loc[family, 'assigned_day'])

    if(day_before==True and day_after==True):
        improvement = (penalty_before-penalty_after)+(accounting_before-accounting_after)
    else:
        improvement = -1
    
    return improvement

In [55]:
family_id = 100
check_swap_day(data, family_id, 0)

3084.460188415094

In [57]:
def check_swap_family(data, family, choice):
    family1 = family
    day_family1 = data.loc[family1, 'assigned_day']
    penalty1 = data.loc[family1, 'penalty_cost']
    member_family1 = data.loc[family1, 'n_people']
    
    day_member_list = data.groupby('assigned_day')['family_id'].apply(list).to_frame()
    
    improvements = {}
    for member in day_member_list.loc[data.loc[family1, 'choice_'+str(choice)], 'family_id']:
        family2 = member
        day_family2 = data.loc[family2, 'assigned_day']
        member_family2 = data.loc[family2, 'n_people']
        penalty2 = data.loc[family2, 'penalty_cost']
        
        # simulate the swap with another family
        data_copy = data.copy()
        data_copy.loc[family2, 'assigned_day'] = data_copy.loc[family1, 'assigned_day']
        data_copy.loc[family1, 'assigned_day'] = data_copy.loc[family1, 'choice_'+str(choice)]
        # calc the new penalty cost for both families
        new_penalty1 = calc_family_costs(data_copy.iloc[family1])
        new_penalty2 = calc_family_costs(data_copy.iloc[family2])
        # check both days before and after swaping
        day_before = check_day(data_copy, data.loc[family1, 'assigned_day'])
        day_after = check_day(data_copy, data_copy.loc[family1, 'choice_'+str(choice)])
        # calc the accounting costs before and after swaping
        accounting_before = calc_accounting_cost(data)
        accounting_after = calc_accounting_cost(data_copy)
        if(day_before==True and day_after==True):
            improvement = (penalty1-new_penalty1) + (penalty2-new_penalty2) + (accounting_before-accounting_after)
        else:
            improvement = -1
        improvements.update({member:improvement})
   
    maximum = max(zip(improvements.values(), improvements.keys()))
    family_swap = maximum[1]
    return improvement, family_swap

In [63]:
data = data.reset_index()
family_id = 100
check_swap_family(data, family_id, 0)

(3099.702689288188, 3161)

In [64]:
family_id = 386
choice = 0
improvement_day = check_swap_day(data, family_id, choice)
improvement_family, family_swap = check_swap_family(data, family_id, choice)
improvement_day, improvement_family, family_swap

(3950.067390674625, 3966.949943189299, 4734)